**Fetch Rewards Coding Exercise**

Pt3 - Data Quality

Section 2 - Nest JSON & Date Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import seaborn as sns

import duckdb

* [Overview](#Overview)
    * [Import Tables & Reivew](#import)
    * [Recommendations for Cleaning](#cleaning)

# Overview <a class="anchor" id="Overview"></a>
**Evaluating (i) nested JSON across the tables and (ii) formats of date-related columns**


Background: 
- Across all 3 tables, they have columns contained nested json fields and have their date values represented in a Unix Timestamp. 

Why This Matters:
- It's vital for final tables viewed by analysts, data scientists, and other data professionals to be as clean as possible. This frees up quering/data cleaning time during analysis projects and ensures data values are represented to the most comprehensvie amnner. 

------------------------

## Import & Review gz files <a id="import"></a>

**Summary:** Across all 3 tables, they have columns contained nested json fields and have their date values represented in a Unix Timestamp. 

In [9]:
df_brands = pd.read_json('brands.gz', lines=True, compression='gzip')
df_receipts = pd.read_json('receipts.gz', lines=True, compression='gzip')
df_users = pd.read_json('users.gz', lines=True, compression='gzip')

In [3]:
df_brands.head(3)

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176


In [4]:
df_receipts.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b


In [5]:
df_users.head(3)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


-------------------------------------


## Cleaning Tables <a id="cleaning"></a>

**df_brands cleaning**

In [ ]:
#Extracting the id value number from _id and renaming it to brand_id
# Suggestion to edit '_id' to say brand_id
df_brands['brand_id'] = pd.json_normalize(df_brands['_id'])['$oid']
df_brands.drop('_id', axis = 1, inplace = True)

#Extracting values from the 'cpg' column and renaming the separate columns
df_cpg_normalized = pd.json_normalize(df_brands['cpg'], sep = '_').rename(columns = {
    '$ref': 'cpg_ref',
    '$id_$oid': 'cpg_id'
})

df_brands['cpg_ref'] = df_cpg_normalized['cpg_ref']
df_brands['cpg_id'] = df_cpg_normalized['cpg_id']
df_brands = df_brands.drop(columns = ['cpg'])

In [11]:
df_brands.head(3)

,barcode,category,categoryCode,name,topBrand,brandCode,brand_id,cpg_ref,cpg_id
0,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,601ac115be37ce2ead437551,Cogs,601ac114be37ce2ead437550
1,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f,Cogs,5332f5fbe4b03c9a25efd0ba
2,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,Cogs,601ac142be37ce2ead437559


**df_receipts cleaning**

In [12]:
#Extract id value number from _id and renaming it to receipt_id
# Suggestion to edit '_id' to receipt_id
df_receipts['receipt_id'] = pd.json_normalize(df_receipts['_id'])['$oid']
df_receipts.drop('_id', axis = 1, inplace = True)


#Identify the date columns 
date_columns = list(set(['createDate', 'finishedDate', 'dateScanned', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']))

#Normalize the column JSON , extract UNIX date value
#Convert UNIX Date value into datetime
for col in date_columns:
    if isinstance(df_receipts[col].iloc[0], dict):
        df_receipts[col] = pd.json_normalize(df_receipts[col])['$date']
    df_receipts[col] = pd.to_datetime(df_receipts[col], unit = 'ms', errors = 'coerce')

**df_users cleaning**

In [13]:
#Extract id value number from _id and renaming it to receipt_id
# Suggestion to edit '_id' to user_id
df_users['user_id'] = pd.json_normalize(df_users['_id'])['$oid']
df_users.drop('_id', axis = 1, inplace = True)

#Normalize date related column JSONs and convert to datetime
df_users['createdDate'] = pd.json_normalize(df_users['createdDate'])['$date']
df_users['lastLogin'] = pd.json_normalize(df_users['lastLogin'])['$date']

df_users['createdDate'] = pd.to_datetime(df_users['createdDate'], unit = 'ms', errors = 'coerce')
df_users['lastLogin'] = pd.to_datetime(df_users['lastLogin'], unit = 'ms', errors= 'coerce')